# Generate word2vec model - include all new topic words

In [1]:
from gensim.models import Word2Vec, KeyedVectors
import gensim
import time
import numpy as np
from tqdm import tqdm
from pprint import pprint
import pickle
import spacy
import re
import operator
import spacy_udpipe

In [2]:
nlp = spacy.load('nl_core_news_sm')

In [2]:
# set variables 
LANG = 'nl'
MAX_WORDS = 500000  # maximum processed words to small model 

MODEL_DIR = '/mnt/data/data/models-fasttext/'
TOPICS_DIR = '/home/dzon/kajo/topics/topic_lists-based_on_en-US/'

model_filename = MODEL_DIR + 'cc.'+ LANG +'.300-lemma_500k_min2.vec.bin'

excluded_model_name = MODEL_DIR + 'cc.'+ LANG +'.300-lemma_500k_min2_excluded.vec.bin'

model_orig_filename = MODEL_DIR + 'cc.'+ LANG +'.300.vec.bin'
model_orig_filename_vec = MODEL_DIR + 'cc.'+ LANG +'.300.vec.gz'

pickled_words_lemmas = MODEL_DIR + LANG + '_ft_all_words_lemmas.pickle'

lemma_list_file = TOPICS_DIR + 'topics_lemma_'+ LANG +'.lst'
nolemma_list_file = TOPICS_DIR + 'topics_'+ LANG +'.lst'

excluded_filename = TOPICS_DIR + 'topics_' + LANG + '-not_in_model.lst'

lemma_list_file_excluded = TOPICS_DIR + 'topics_lemma_excluded_' + LANG + '.lst'
nolemma_list_file_excluded = TOPICS_DIR + 'topics_excluded_' + LANG + '.lst'


In [4]:
# load the big model
model_big = KeyedVectors.load_word2vec_format(model_orig_filename, binary=True)

In [5]:
# process all words in fasttext model  takes ~3h wirh spaCy model (~10min with UDPIPE model)
all_words_lemmas = []

lemmas = {}
word_lemma = {}
words = model_big.index2word
for word in tqdm(words):
    # skip "words" which are not regular words
    if not re.fullmatch("([^\d\W])+", word):
        continue
    doc = nlp(word)
    lemma = doc[0].lemma_.lower()
    all_words_lemmas.append((word, lemma))
print("Number of words in big model: ", len(all_words_lemmas))

#save pickled dump 
with open(pickled_words_lemmas, 'wb') as f:
    pickle.dump(all_words_lemmas, f)

100%|██████████| 2000000/2000000 [3:43:15<00:00, 149.31it/s]  


Number of words in big model:  1591867


In [6]:
#load pickled lemma list
with open(pickled_words_lemmas, 'rb') as f:
    all_words_lemmas = pickle.load(f)
pprint(all_words_lemmas[:50])

[('de', 'de'),
 ('van', 'van'),
 ('en', 'en'),
 ('een', 'een'),
 ('het', 'het'),
 ('in', 'in'),
 ('is', 'zijn'),
 ('op', 'op'),
 ('voor', 'voor'),
 ('met', 'met'),
 ('De', 'de'),
 ('te', 'te'),
 ('zijn', 'zijn'),
 ('dat', 'dat'),
 ('je', 'je'),
 ('die', 'die'),
 ('door', 'door'),
 ('aan', 'aan'),
 ('niet', 'niet'),
 ('of', 'of'),
 ('uit', 'uit'),
 ('om', 'om'),
 ('als', 'als'),
 ('Het', 'het'),
 ('ook', 'ook'),
 ('bij', 'bij'),
 ('naar', 'naar'),
 ('er', 'er'),
 ('was', 'zijn'),
 ('ik', 'ik'),
 ('werd', 'worden'),
 ('In', 'in'),
 ('wordt', 'worden'),
 ('maar', 'maar'),
 ('deze', 'deze'),
 ('tot', 'tot'),
 ('hij', 'hij'),
 ('worden', 'worden'),
 ('dan', 'dan'),
 ('over', 'over'),
 ('heeft', 'hebben'),
 ('nog', 'nog'),
 ('s', 's'),
 ('dit', 'dit'),
 ('kan', 'kunnen'),
 ('ze', 'ze'),
 ('meer', 'me'),
 ('u', 'u'),
 ('naam', 'naam'),
 ('wat', 'wat')]


In [7]:
# generate helpful dictionaries/lists
big_lemmas_set = set([l for (_,l) in all_words_lemmas])
print(len(big_lemmas_set))

big_words_set = set([w for (w,_) in all_words_lemmas])
print(len(big_words_set))

lemmas = {}
words_lower_orig = {}
big_lemma_word_dic = {}
big_word_lemma_dic = {}
for (w,l) in all_words_lemmas:
    big_word_lemma_dic[w] = l
    
    #dictionary of lowercase word -> all original words
    if w.lower() in words_lower_orig:
        words_lower_orig[w.lower()].append(w)
    else:
        words_lower_orig[w.lower()] = [w]
        
    if l in big_lemma_word_dic:
        big_lemma_word_dic[l].append(w)
        lemmas[l] += 1
    else:
        big_lemma_word_dic[l] = [w]
        lemmas[l] = 1
        
# load all the words from big model
words = model_big.wv.index2word
words_lower = [w.lower() for w in words]

#words_lemma = model_lemma_big.index2word       

1169653
1591867


/home/dzon/kajo/kajo_semexp_core/lib/python3.6/site-packages/ipykernel_launcher.py:29: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).


In [8]:
# sort the lemmas from most frequent
sorted_lemmas = sorted(lemmas.items(), key=operator.itemgetter(1), reverse=True)
# trying first words
sorted_lemmas[:20]

[('zijn', 42),
 ('hebben', 37),
 ('goed', 33),
 ('gaan', 30),
 ('worden', 29),
 ('pop', 28),
 ('komen', 27),
 ('kijken', 27),
 ('doen', 26),
 ('willen', 26),
 ('leven', 26),
 ('huis', 25),
 ('man', 25),
 ('eten', 25),
 ('denken', 25),
 ('horen', 25),
 ('kunnen', 24),
 ('volgen', 24),
 ('geven', 24),
 ('snel', 24)]

In [9]:
# create vectors for the small models - average vectors for lemmas
# process only first MAX_WORDS in the big model
tmp_words = [None] * len(model_big.wv.index2word)
tmp_vecs = [None] * len(model_big.wv.index2word) 
for i,(word,lemma) in enumerate(all_words_lemmas[:MAX_WORDS]):
    lemma_words_list = big_lemma_word_dic[lemma]
    # add only lemmas made from at least 2 words
    if len(lemma_words_list)<2:
        continue
    tmp_vectors = []
    tmp_indexes = []
    error_with_word = False
    for word2 in lemma_words_list:
        try:
            tmp_vectors.append(model_big[word2])  
            tmp_indexes.append(model_big.vocab[word2].index)
        except:
            print(i, "error with ", word, " lemma ", lemma, "words", lemma_words_list)
            error_with_word = True
            continue
    if not error_with_word:
        try:
            tmp_words[min(tmp_indexes)] = lemma
            tmp_vecs[min(tmp_indexes)] = list(np.average(tmp_vectors, axis=0))
        except:
            print(i, "error with lemma ", lemma, " - words", lemma_words_list, " indexes: ", tmp_indexes)
            # note - we are processing first MAX_WORDS from the model after the filter out of the "non-words"
            # so some of the last words in that range may refer to words on bigger index than MAX_WORDS

# filter the lists (some positions remained None)
final_words = [i for i in tmp_words if i] 
final_vecs = [i for i in tmp_vecs if i != None] 

print(len(final_vecs), len(final_words))
len(final_vecs[0])  

del tmp_vecs
del tmp_words

/home/dzon/kajo/kajo_semexp_core/lib/python3.6/site-packages/ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  This is separate from the ipykernel package so we can avoid doing imports until
/home/dzon/kajo/kajo_semexp_core/lib/python3.6/site-packages/ipykernel_launcher.py:4: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  after removing the cwd from sys.path.


165945 165945


In [10]:
# create new gensim word2vec model with the vectors:
new_vector_space = gensim.models.keyedvectors.Word2VecKeyedVectors(300)
new_vector_space.add(final_words,final_vecs)

In [13]:
# test the model
pprint(new_vector_space.most_similar('vader'))
pprint(new_vector_space.most_similar_cosmul(positive=['vrouw', 'koning'], negative=['man']))

[('moeder', 0.7843632698059082),
 ('zoon', 0.7707353830337524),
 ('grootvader', 0.7188552618026733),
 ('dochter', 0.7042664289474487),
 ('vaderlijk', 0.6708673238754272),
 ('kleinzoon', 0.633141279220581),
 ('broer', 0.6265909671783447),
 ('vaderschap', 0.6233826875686646),
 ('echtgenoot', 0.6167832016944885),
 ('kind', 0.6116095781326294)]
[('koningin', 0.925083577632904),
 ('koningsdochter', 0.8144248723983765),
 ('koningschap', 0.814302921295166),
 ('koningszoon', 0.8088672757148743),
 ('koningskind', 0.8056352734565735),
 ('dienaar', 0.7982543706893921),
 ('koningshuis', 0.7980857491493225),
 ('prinses', 0.7964479327201843),
 ('koninginnentitel', 0.795605480670929),
 ('echtgenoot', 0.7949511408805847)]


In [14]:
# save the model
new_vector_space.save_word2vec_format(model_filename, binary=True)

In [15]:
#model = KeyedVectors.load_word2vec_format(model_filename, binary=True)
model = new_vector_space
words_model = model.wv.index2word

/home/dzon/kajo/kajo_semexp_core/lib/python3.6/site-packages/ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  This is separate from the ipykernel package so we can avoid doing imports until


In [1]:
# load topic listst
with open(nolemma_list_file) as fp:
    topics_nolemma = fp.readlines() 
print(len(topics_nolemma))
    
# load the lemma list
# with open(lemma_list_file) as fp:
#    topics = fp.readlines()
# print(len(topics))

topics = []
# generate the lemma list
for topic in tqdm(topics_nolemma):
    topic = re.sub('\W',' ',topic)
    topic = re.sub('\ +',' ',topic)
    doc = nlp(topic.strip().lower())
    lemmas = []
        
    for token in doc:
        if token.is_stop:
            continue
        lemmas.append(token.lemma_)    
    if len(lemmas) == 0:
        print('lemmas = 0 ', topic)
    topics.append("_".join(lemmas))
    
print(len(topics))    
    
#write the lemma topic list
with open(lemma_list_file, 'wt', encoding='utf8') as f:
    for topic_lemma in topics:
        f.write(topic_lemma + "\n")    

NameError: name 'nolemma_list_file' is not defined

In [17]:
print(len(topics),len(topics_nolemma))  

5308 5308


In [18]:
#try if the topic-lemma words exist in the models  

words_to_manual_add = {}

for i, (topic, topic_nolemma) in enumerate(zip(topics, topics_nolemma)):
    # process the nolemma topic so we can see which word was lemmatized to which lemma
    topic_nolemma = re.sub('\W',' ',topic_nolemma)
    topic_nolemma = re.sub('\ +',' ',topic_nolemma)
    doc = nlp(topic_nolemma.strip().lower())
    nolemmas = []
    for token in doc:
        if token.is_stop:
            continue
        else:
            nolemmas.append(token.text)  
    
    topic = topic.strip()
    topic_words = topic.split('_')
    if len(topic_words) != len(nolemmas):
        print("!!!!! error - not equal length: ", nolemmas, topic_words)
        continue
    for word, word_nolemma in zip(topic_words, nolemmas):
        #print(word)
        found = 0
        if word in words_model:
            found = 1
            #print("l1")
        elif word in big_lemmas_set:
            found = 1
            print("l2", word)
        elif word in words_lower:
            found = 1
            print("l3", word)
        # nolemma words are in PL case probably not needed and all lemmas exist in lemma list
        elif word_nolemma in words_model:
            found = 1
            print("w1", word)
        elif word_nolemma in big_lemmas_set:
            found = 1
            print("w2", word)
        elif word_nolemma in words_lower:
            found = 1
            print("w3", word)

        if found == 0:
            print(i, 'needs to be added manually: ', word, " - ", word_nolemma, " - ", topic_nolemma.strip())
            if word not in words_to_manual_add:
                words_to_manual_add[word] = topic_nolemma.strip()
                    
print(len(words_to_manual_add))          

l2 landbouwlandschap
l2 landschapsplanning
25 needs to be added manually:  grondgebruiksplanning  -  grondgebruiksplanning  -  grondgebruiksplanning
l2 plattelandsbevolking
37 needs to be added manually:  midelbaar  -  midelbaar  -  midelbaar onderwijs
l2 bodemverdichting
l2 bodemdegradatie
l2 waterlichaam
l2 winderosie
48 needs to be added manually:  kustecosysteem  -  kustecosysteem  -  kustecosysteem
49 needs to be added manually:  zoetwaterecosysteem  -  zoetwaterecosysteem  -  zoetwaterecosysteem
52 needs to be added manually:  grondgebruiksplan  -  grondgebruiksplan  -  grondgebruiksplan
54 needs to be added manually:  binnenwatervervoer  -  binnenwatervervoer  -  binnenwatervervoer
55 needs to be added manually:  landbouwprij  -  landbouwprijs  -  landbouwprijs
l2 milieuverandering
l2 landbeheer
62 needs to be added manually:  waterbronnenbescherming  -  waterbronnenbescherming  -  waterbronnenbescherming
63 needs to be added manually:  motorvoertuigenindustrie  -  motorvoertuig

560 needs to be added manually:  zoetwatervervuiling  -  zoetwatervervuiling  -  zoetwatervervuiling
l2 pluimveefokkerij
562 needs to be added manually:  bodemcapaciteit  -  bodemcapaciteit  -  bodemcapaciteit
l2 milieumarkt
l2 landbouwgebouw
l2 milieuveiligheid
w2 bevolkingstren
l2 productiebeleid
589 needs to be added manually:  koelindustrie  -  koelindustrie  -  koelindustrie
592 needs to be added manually:  landbouwongedierte  -  landbouwongedierte  -  landbouwongedierte
l2 scheepswerktuigkunen
l2 ontwikkelingspatroon
l2 bodembiologie
606 needs to be added manually:  bodemstabiliteit  -  bodemstabiliteit  -  bodemstabiliteit
607 needs to be added manually:  bevloeiingslandbouw  -  bevloeiingslandbouw  -  bevloeiingslandbouw
l2 waterbescherming
614 needs to be added manually:  milieudoorlichting  -  milieudoorlichting  -  milieudoorlichting
l2 brakwater
l2 stadsstructuur
l2 landschapsherstel
l2 stralingseffect
628 needs to be added manually:  zoetwaterorganisme  -  zoetwaterorganis

1093 needs to be added manually:  milieugeneeskunen  -  milieugeneeskunde  -  milieugeneeskunde
1096 needs to be added manually:  verbruikersbescherming  -  verbruikersbescherming  -  verbruikersbescherming
l2 huisvestingsprogramma
l2 industriebond
l2 gezondheids
1113 needs to be added manually:  dienstverleningsgebied  -  dienstverleningsgebied  -  dienstverleningsgebied
1125 needs to be added manually:  milieupsychologie  -  milieupsychologie  -  milieupsychologie
1130 needs to be added manually:  stadsdruk  -  stadsdruk  -  stadsdruk
l2 bouwonderdeel
l2 soortbescherming
l2 milieubesef
l2 kooldioxien
1140 needs to be added manually:  zoetwatermeting  -  zoetwatermeting  -  zoetwatermeting
1148 needs to be added manually:  bosbouwwetgeving  -  bosbouwwetgeving  -  bosbouwwetgeving
1149 needs to be added manually:  bosaanplanting  -  bosaanplanting  -  plaatselijke bosaanplanting
1151 needs to be added manually:  afvalwetgeving  -  afvalwetgeving  -  afvalwetgeving
1153 needs to be add

1591 needs to be added manually:  landbouwgrondbezit  -  landbouwgrondbezit  -  landbouwgrondbezit
1597 needs to be added manually:  bedrijfsveiligheidsvoorschrift  -  bedrijfsveiligheidsvoorschrift  -  bedrijfsveiligheidsvoorschrift
l2 milieukwaliteitsnorm
1604 needs to be added manually:  afvalwaterkwaliteit  -  afvalwaterkwaliteit  -  afvalwaterkwaliteit
l2 eigendomsregister
l2 afvalwaterverwerking
l2 milieuhygiëne
1621 needs to be added manually:  internetzoekoptie  -  internetzoekoptie  -  Internetzoekoptie
1622 needs to be added manually:  aardgasexploratie  -  aardgasexploratie  -  aardgasexploratie
1623 needs to be added manually:  milieubeheerindicator  -  milieubeheerindicator  -  milieubeheerindicator
l2 landbouwwater
1626 needs to be added manually:  milieukwaliteitsdoelstelling  -  milieukwaliteitsdoelstelling  -  milieukwaliteitsdoelstelling
1627 needs to be added manually:  afvalwaterbelasting  -  afvalwaterbelasting  -  afvalwaterbelasting
l2 afvalwaterverwerking
1633 n

1862 needs to be added manually:  populatiestatistiek  -  populatiestatistieken  -  populatiestatistieken over in het wild levende planten en dieren
l2 bouwnorm
1868 needs to be added manually:  verwijzingsinformatiesysteem  -  verwijzingsinformatiesysteem  -  verwijzingsinformatiesysteem
l2 beheersprogramma
1872 needs to be added manually:  luchtverkeerswetgeving  -  luchtverkeerswetgeving  -  luchtverkeerswetgeving
l2 loodgehalte
1877 needs to be added manually:  bedrijfsorgaan  -  bedrijfsorgaan  -  publiekrechtelijk bedrijfsorgaan
l2 milieuplan
1882 needs to be added manually:  afvalverwijderingsindustrie  -  afvalverwijderingsindustrie  -  afvalverwijderingsindustrie
1883 needs to be added manually:  drinkwaterbeschermingsgebied  -  drinkwaterbeschermingsgebied  -  drinkwaterbeschermingsgebied
1886 needs to be added manually:  afvalwaterstatistieken  -  afvalwaterstatistieken  -  afvalwaterstatistieken
l2 tweedehandsgoeder
1896 needs to be added manually:  immissiebeheersingswet  

l2 plantenecologie
l2 bedrijfsgeneeskunen
2538 needs to be added manually:  grasbrand  -  grasbrand  -  grasbrand
2541 needs to be added manually:  estuariumvervuiling  -  estuariumvervuiling  -  estuariumvervuiling
2547 needs to be added manually:  toeristenvoorziening  -  toeristenvoorziening  -  toeristenvoorziening
l2 gegevenscentrum
2556 needs to be added manually:  bouwuitrusting  -  bouwuitrusting  -  bouwuitrusting
l2 verenigde
l3 naties
2567 needs to be added manually:  foktechniek  -  foktechniek  -  foktechniek
l2 controlenorm
2573 needs to be added manually:  slibstabilisatie  -  slibstabilisatie  -  slibstabilisatie
2575 needs to be added manually:  waarschuwingsplan  -  waarschuwingsplan  -  waarschuwingsplan
2576 needs to be added manually:  volgplan  -  volgplan  -  volgplan
l2 weginrichting
l2 seizoensmigratie
2595 needs to be added manually:  stranspiratie  -  stranspiratie  -  verdamping stranspiratie
2596 needs to be added manually:  zeewatercultuur  -  zeewatercult

3182 needs to be added manually:  soortverarming  -  soortverarming  -  soortverarming
3184 needs to be added manually:  landschapsherkenning  -  landschapsherkenning  -  landschapsherkenning
l3 warmt
3191 needs to be added manually:  giftigheidsboordeling  -  giftigheidsboordeling  -  giftigheidsboordeling
l2 afvalolie
l2 kristalliser
l2 woestijnsprinkhaan
3206 needs to be added manually:  verstooiing  -  verstooiing  -  verstooiing van verontreinigende stoffen
l2 afvalsortering
l2 mijnbouwproduct
3221 needs to be added manually:  cementvervaardiging  -  cementvervaardiging  -  cementvervaardiging
3223 needs to be added manually:  buikvoetig  -  buikvoetigen  -  buikvoetigen
l3 0
3232 needs to be added manually:  mededingingsregeling  -  mededingingsregeling  -  mededingingsregeling
l2 sportvoorziening
l2 metaalerts
l2 visplaats
3237 needs to be added manually:  machinetrilling  -  machinetrilling  -  machinetrilling
3241 needs to be added manually:  voedingsbalans  -  voedingsbalans 

l2 plantendeel
l2 zonnekrachtcentrale
3762 needs to be added manually:  omkeringslaag  -  omkeringslaag  -  omkeringslaag
3765 needs to be added manually:  monsternametechniek  -  monsternametechniek  -  monsternametechniek
3767 needs to be added manually:  ontchloring  -  ontchloring  -  ontchloring
3770 needs to be added manually:  dispersieberekening  -  dispersieberekening  -  dispersieberekening
l2 halfgeleid
l2 kleursamenstelling
3781 needs to be added manually:  smogwaarschuwing  -  smogwaarschuwing  -  smogwaarschuwing
3782 needs to be added manually:  immissiebeheersing  -  immissiebeheersing  -  immissiebeheersing
l2 kernbrandstof
l2 glasafval
3792 needs to be added manually:  remmingsbekken  -  remmingsbekken  -  remmingsbekken
l2 uitrustingsprogramma
l2 composteerbaar
l2 meettoestell
l2 aanpassingsperiode
3811 needs to be added manually:  loodverontreiniging  -  loodverontreiniging  -  loodverontreiniging
3814 needs to be added manually:  zoonosi  -  zoonosis  -  zoonosis
l

4318 needs to be added manually:  ammonificatie  -  ammonificatie  -  ammonificatie
l2 invoervergunning
l2 hydrobiologie
4330 needs to be added manually:  afvalinvoervergunning  -  afvalinvoervergunning  -  afvalinvoervergunning
4338 needs to be added manually:  zoutlading  -  zoutlading  -  zoutlading
l2 kanaalsluis
4355 needs to be added manually:  fosfaatvervangingsmiddel  -  fosfaatvervangingsmiddel  -  fosfaatvervangingsmiddel
4361 needs to be added manually:  beeldclassificatie  -  beeldclassificatie  -  beeldclassificatie
4362 needs to be added manually:  beeldclassificatie  -  beeldclassificatie  -  beeldclassificatie zonder toezicht
l2 oxyderenen
l2 embryogenese
4367 needs to be added manually:  elektrokinetica  -  elektrokinetica  -  elektrokinetica
4369 needs to be added manually:  afvalverwerkingsheffing  -  afvalverwerkingsheffing  -  afvalverwerkingsheffing
l2 kernongevall
4382 needs to be added manually:  droogontsmetting  -  droogontsmetting  -  droogontsmetting van zaa

l2 geluidssterken
l2 galvanisatie
l2 verzendbericht
l2 naverbranding
l2 ozonisatie
5068 needs to be added manually:  runderachtig  -  runderachtigen  -  runderachtigen
5071 needs to be added manually:  aërobiologie  -  aërobiologie  -  aërobiologie
l2 paleoklimatologie
5078 needs to be added manually:  reflectometrie  -  reflectometrie  -  reflectometrie
l2 alveole
l2 ongedieren
l2 basidiomycete
5094 needs to be added manually:  verambtelijking  -  verambtelijking  -  verambtelijking
5095 needs to be added manually:  polychloordibenzo  -  polychloordibenzo  -  polychloordibenzo p dioxine
5100 needs to be added manually:  polybroombifenyl  -  polybroombifenyl  -  polybroombifenyl
l2 polychloorbifenyl
5103 needs to be added manually:  bederfweren  -  bederfwerend  -  bederfwerend middel
5108 needs to be added manually:  algendodenen  -  algendodende  -  algendodende stof
l2 colorimetrie
l2 cyanaat
l2 vulcaniseren
5123 needs to be added manually:  olfactometrie  -  olfactometrie  -  olfac

In [19]:
# for manual correcting of words
pprint(words_to_manual_add)

{'aansprakelijkheidswetgeving': 'aansprakelijkheidswetgeving',
 'aanvangstraining': 'aanvangstraining',
 'aardgasexploratie': 'aardgasexploratie',
 'aardoliebronn': 'bescherming van de aardoliebronnen',
 'aardolieramp': 'aardolieramp',
 'abondantie': 'ecologische abondantie',
 'accumulatief': 'bio accumulatieve verontreinigende stof',
 'activiteitsrelatie': 'structuur activiteitsrelatie',
 'adsorbeerbar': 'gehalte aan adsorbeerbare organohalogenen AOX',
 'afbakeningsprocedure': 'afbakeningsprocedure',
 'afdekstro': 'afdekstro',
 'afgravingplaat': 'afgravingplaats',
 'afgravingskant': 'afgravingskant',
 'afstandsverwarming': 'afstandsverwarming',
 'afvalanalyse': 'afvalanalyse',
 'afvalbalan': 'afvalbalans',
 'afvalbeperking': 'afvalbeperking',
 'afvalcategorie': 'afvalcategorie',
 'afvalclassificatie': 'afvalclassificatie',
 'afvalgassenverspreiding': 'afvalgassenverspreiding',
 'afvalgebruik': 'afvalgebruik',
 'afvalinvoervergunning': 'afvalinvoervergunning',
 'afvalomzettingstechniek

In [20]:
# the lemmas not in model were manually checked if they exist in other models in some variation
word = "φυτοκομείο"
print(word)
try:
    model[word]
    print('model')
    pprint(model.most_similar(word))
except:
    pass

if word in big_lemmas_set:
    print('model_lemma_big', big_lemma_word_dic[word])

try:
    model_big[word]
    print('model big')
    pprint(model_big.most_similar(word))
except:
    pass

# if the original word with uppercase
if word in words_lower_orig:
    print('model big LOWER', words_lower_orig[word])
    pprint(model_big.most_similar(words_lower_orig[word]))
    

φυτοκομείο


In [21]:
# for cases the lemmas aren't correct or don't exist in the dic
# manually made dictionary of "lemma" : "word which exists in some model" or "word1;word2" for the average
# e.g. 'mikroekosystém': 'mikro;ekosystém', 'bahný': 'bahno'
new_words_manually_checked = {}
    
len(new_words_manually_checked)

0

In [22]:
# function which finds the word in all the available models and returns it's vector (average vector)
def get_vector(word):
    # word is in small model
    try:
        return model[word]
    except:
        pass
    
    # word is in lemmas from big model - return average if the lemma was result of more words
    if word in big_lemmas_set:
        lemma_words_list = big_lemma_word_dic[word]
        tmp_vectors = []
        for word2 in lemma_words_list:
            tmp_vectors.append(model_big[word2])  
        return np.average(tmp_vectors, axis=0)

    try:
        return model_big[word]
    except:
        pass
    
    # if the original word with uppercase
    if word in words_lower_orig:
        orig_words_list = words_lower_orig[word]
        tmp_vectors = []
        for word2 in orig_words_list:
            tmp_vectors.append(model_big[word2])  
        return np.average(tmp_vectors, axis=0)

        #return model_big[words_lower_orig[word]]
    
    return []

In [23]:
# check if the word vectors are the same
#get_vector('Antarktyki') == get_vector('antarktyki')
get_vector('διβενζοφουράνιο')

[]

In [24]:
# ading words from manually checked list
new_vectors = []
new_words = []
for i, word in enumerate(new_words_manually_checked.keys()):
    
    tmp_vectors = []
    print(i, word)
    for tmp_word in new_words_manually_checked[word].split(';'):
        vector = get_vector(tmp_word)
        if vector != []:
            tmp_vectors.append(vector)
        else:
            print("XXX - some ERROR, word doesn't have vector, cannot add: ", tmp_word)
    if word not in new_words:
        new_vectors.append(np.average(tmp_vectors, axis=0))
        new_words.append(word)
print(len(new_vectors),len(new_words))        

0 0


In [25]:
# add new words to the model
if len(new_words) > 0:
    model.add(new_words,new_vectors)

In [26]:
print(len(topics),len(topics_nolemma))  

5308 5308


In [27]:
# 2nd run - checking if there are still unavailable words in topics
excluded_topics = []
#try if the topic-lemma words exist in the model   

for i, (topic, topic_nolemma) in enumerate(zip(topics, topics_nolemma)):
    # process the nolemma topic so we can see which word was lemmatized to which lemma
    topic_nolemma_orig = topic_nolemma.strip()
    topic_nolemma = re.sub('\W',' ',topic_nolemma)
    topic_nolemma = re.sub('\ +',' ',topic_nolemma)
    doc = nlp(topic_nolemma.strip().lower())
    nolemmas = []
    for token in doc:
        if token.is_stop:
            continue
        else:
            nolemmas.append(token.text)  
    
    topic_words = topic.split('_')
    
    if len(topic_words) != len(nolemmas):
        print("!!!!! error - not equal length: ", nolemmas, topic_words)
        continue
        
    found = 1
    for word, word_nolemma in zip(topic_words, nolemmas):
    #for word in topic_words:
        #print(word)
       
        vector = get_vector(word)
        if vector == []:
            print('nonexistend word: ', word, " - ", topic_nolemma)
            vector2 = get_vector(word_nolemma)
            if vector2 == []:
                found = 0
                print('nonexistend word: ', word_nolemma, " - ", topic_nolemma)
            else:
                print("Exist nolemma")

    if found == 0:
        print('!!! topic needs to be excluded: ', topic_nolemma_orig)
        if topic_nolemma.strip() not in excluded_topics:
            excluded_topics.append(topic_nolemma_orig)
                    
print(len(excluded_topics)) 

nonexistend word:  grondgebruiksplanning  -  grondgebruiksplanning 
nonexistend word:  grondgebruiksplanning  -  grondgebruiksplanning 
!!! topic needs to be excluded:  grondgebruiksplanning


/home/dzon/kajo/kajo_semexp_core/lib/python3.6/site-packages/ipykernel_launcher.py:30: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.


nonexistend word:  midelbaar  -  midelbaar onderwijs 
nonexistend word:  midelbaar  -  midelbaar onderwijs 
!!! topic needs to be excluded:  midelbaar onderwijs
nonexistend word:  kustecosysteem  -  kustecosysteem 
nonexistend word:  kustecosysteem  -  kustecosysteem 
!!! topic needs to be excluded:  kustecosysteem
nonexistend word:  zoetwaterecosysteem  -  zoetwaterecosysteem 
nonexistend word:  zoetwaterecosysteem  -  zoetwaterecosysteem 
!!! topic needs to be excluded:  zoetwaterecosysteem
nonexistend word:  grondgebruiksplan  -  grondgebruiksplan 
nonexistend word:  grondgebruiksplan  -  grondgebruiksplan 
!!! topic needs to be excluded:  grondgebruiksplan
nonexistend word:  binnenwatervervoer  -  binnenwatervervoer 
nonexistend word:  binnenwatervervoer  -  binnenwatervervoer 
!!! topic needs to be excluded:  binnenwatervervoer
nonexistend word:  landbouwprij  -  landbouwprijs 
nonexistend word:  landbouwprijs  -  landbouwprijs 
!!! topic needs to be excluded:  landbouwprijs
nonex

nonexistend word:  beleidsintegratie  -  beleidsintegratie 
nonexistend word:  beleidsintegratie  -  beleidsintegratie 
!!! topic needs to be excluded:  beleidsintegratie
nonexistend word:  voedselvervo  -  voedselvervoer 
nonexistend word:  voedselvervoer  -  voedselvervoer 
!!! topic needs to be excluded:  voedselvervoer
nonexistend word:  stadsvoorziening  -  stadsvoorziening 
nonexistend word:  stadsvoorziening  -  stadsvoorziening 
!!! topic needs to be excluded:  stadsvoorziening
nonexistend word:  stadsvervuiler  -  stadsvervuiler 
nonexistend word:  stadsvervuiler  -  stadsvervuiler 
!!! topic needs to be excluded:  stadsvervuiler
nonexistend word:  dierenfysiologie  -  dierenfysiologie 
nonexistend word:  dierenfysiologie  -  dierenfysiologie 
!!! topic needs to be excluded:  dierenfysiologie
nonexistend word:  luchtruimtelijk  -  luchtruimtelijke planning 
nonexistend word:  luchtruimtelijke  -  luchtruimtelijke planning 
!!! topic needs to be excluded:  luchtruimtelijke plan

/home/dzon/kajo/kajo_semexp_core/lib/python3.6/site-packages/ipykernel_launcher.py:33: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.


...

nonexistend word:  kringloopplastic  -  kringloopplastic 
nonexistend word:  kringloopplastic  -  kringloopplastic 
!!! topic needs to be excluded:  kringloopplastic
nonexistend word:  afvalanalyse  -  afvalanalyse 
nonexistend word:  afvalanalyse  -  afvalanalyse 
!!! topic needs to be excluded:  afvalanalyse
nonexistend word:  delfstofafzetting  -  delfstofafzetting 
nonexistend word:  delfstofafzetting  -  delfstofafzetting 
!!! topic needs to be excluded:  delfstofafzetting
nonexistend word:  heffingsonderscheid  -  heffingsonderscheid 
nonexistend word:  heffingsonderscheid  -  heffingsonderscheid 
!!! topic needs to be excluded:  heffingsonderscheid
nonexistend word:  productvoorlichting  -  productvoorlichting 
nonexistend word:  productvoorlichting  -  productvoorlichting 
!!! topic needs to be excluded:  productvoorlichting
nonexistend word:  noodhulpmaatregel  -  noodhulpmaatregel 
nonexistend word:  noodhulpmaatregel  -  noodhulpmaatregel 
!!! topic needs to be excluded:  no

nonexistend word:  corrosiekracht  -  chemische corrosiekracht 
nonexistend word:  corrosiekracht  -  chemische corrosiekracht 
!!! topic needs to be excluded:  chemische corrosiekracht
nonexistend word:  silicoonverbinding  -  organische silicoonverbinding 
nonexistend word:  silicoonverbinding  -  organische silicoonverbinding 
!!! topic needs to be excluded:  organische silicoonverbinding
nonexistend word:  schademinimalisering  -  schademinimalisering 
nonexistend word:  schademinimalisering  -  schademinimalisering 
!!! topic needs to be excluded:  schademinimalisering
nonexistend word:  tijdstoewijzing  -  tijdstoewijzing 
nonexistend word:  tijdstoewijzing  -  tijdstoewijzing 
!!! topic needs to be excluded:  tijdstoewijzing
nonexistend word:  informatiedistributiecentrum  -  informatiedistributiecentrum 
nonexistend word:  informatiedistributiecentrum  -  informatiedistributiecentrum 
!!! topic needs to be excluded:  informatiedistributiecentrum
nonexistend word:  accumulatief 

nonexistend word:  afvalverwerkingswet  -  afvalverwerkingswet 
nonexistend word:  afvalverwerkingswet  -  afvalverwerkingswet 
!!! topic needs to be excluded:  afvalverwerkingswet
nonexistend word:  restslib  -  restslib 
nonexistend word:  restslib  -  restslib 
!!! topic needs to be excluded:  restslib
nonexistend word:  afvalsorteringseenheid  -  afvalsorteringseenheid 
nonexistend word:  afvalsorteringseenheid  -  afvalsorteringseenheid 
!!! topic needs to be excluded:  afvalsorteringseenheid
nonexistend word:  verwijzingsinformatie  -  verwijzingsinformatie 
nonexistend word:  verwijzingsinformatie  -  verwijzingsinformatie 
!!! topic needs to be excluded:  verwijzingsinformatie
nonexistend word:  microklimatologie  -  microklimatologie 
nonexistend word:  microklimatologie  -  microklimatologie 
!!! topic needs to be excluded:  microklimatologie
nonexistend word:  farmacokinetica  -  farmacokinetica 
nonexistend word:  farmacokinetica  -  farmacokinetica 
!!! topic needs to be e

nonexistend word:  drijfnetvisserij  -  drijfnetvisserij 
nonexistend word:  drijfnetvisserij  -  drijfnetvisserij 
!!! topic needs to be excluded:  drijfnetvisserij
nonexistend word:  blootstellingsgren  -  toegestane blootstellingsgrens 
nonexistend word:  blootstellingsgrens  -  toegestane blootstellingsgrens 
!!! topic needs to be excluded:  toegestane blootstellingsgrens
nonexistend word:  oliebindenen  -  oliebindende stof 
nonexistend word:  oliebindende  -  oliebindende stof 
!!! topic needs to be excluded:  oliebindende stof
nonexistend word:  colubrid  -  Colubriden 
nonexistend word:  colubriden  -  Colubriden 
!!! topic needs to be excluded:  Colubriden
nonexistend word:  verfkamer  -  verfkamer 
nonexistend word:  verfkamer  -  verfkamer 
!!! topic needs to be excluded:  verfkamer
nonexistend word:  metingscriterium  -  metingscriterium 
nonexistend word:  metingscriterium  -  metingscriterium 
!!! topic needs to be excluded:  metingscriterium
nonexistend word:  belastings

nonexistend word:  inventarisatiegid  -  inventarisatiegids emissie van luchtverontreinigende stoffen 
nonexistend word:  inventarisatiegids  -  inventarisatiegids emissie van luchtverontreinigende stoffen 
nonexistend word:  luchtverontreinigend  -  inventarisatiegids emissie van luchtverontreinigende stoffen 
Exist nolemma
!!! topic needs to be excluded:  inventarisatiegids emissie van luchtverontreinigende stoffen
nonexistend word:  zwemwaterwetgeving  -  zwemwaterwetgeving 
nonexistend word:  zwemwaterwetgeving  -  zwemwaterwetgeving 
!!! topic needs to be excluded:  zwemwaterwetgeving
nonexistend word:  samenwerkingsconsumptie  -  samenwerkingsconsumptie 
nonexistend word:  samenwerkingsconsumptie  -  samenwerkingsconsumptie 
!!! topic needs to be excluded:  samenwerkingsconsumptie
nonexistend word:  akkerlandbeheer  -  akkerlandbeheer 
nonexistend word:  akkerlandbeheer  -  akkerlandbeheer 
!!! topic needs to be excluded:  akkerlandbeheer
nonexistend word:  rampenrisico  -  rampe

In [28]:
# save topics containing words which are not present neither model - to be excluded from the topic list
print('excluded topics: ', len(excluded_topics))
pprint(excluded_topics)
with open(excluded_filename, 'wt', encoding='utf8') as f:
    for topic in excluded_topics:
        f.write(topic + "\n")

excluded topics:  721
['grondgebruiksplanning',
 'midelbaar onderwijs',
 'kustecosysteem',
 'zoetwaterecosysteem',
 'grondgebruiksplan',
 'binnenwatervervoer',
 'landbouwprijs',
 'waterbronnenbescherming',
 'motorvoertuigenindustrie',
 'plattelandsbeheer en planning',
 'oppervlaktewaterbeheer',
 'natuurbehoudsprogramma',
 'natuurbehoudswetgeving',
 'landbouwproductiemiddelen',
 'milieubeschermingsverordening',
 'grondgebruiksregime',
 'duurzame ontwikkelingsindicator',
 'kustmilieu',
 'bodemverzilting',
 'plantegroei',
 'watertoezicht',
 'bodemuitloging',
 'landomgeving',
 'gewassenteelt',
 'kredietbijstand',
 'terreinsanering',
 'bosproductie',
 'landbouwplanning',
 'milieuonderhoud',
 'geweststructuur',
 'beveiligingsstelsel',
 'zoetwaterdegradatie',
 'landbouwopslag',
 'voedselhandel',
 'dierenhuisvesting',
 'bodembemesting',
 'landherstel',
 'landbouwuitrusting',
 'landbouwbeheer',
 'voedselbewaring',
 'bodemmineralogie',
 'zoetwaterbescherming',
 'landschapsgebruik',
 'grondverhar

In [29]:
# create new lists which don't containt topics with non-existent words

# load list with excluded topics
with open(excluded_filename, 'rt', encoding='utf8') as fp:
    tmp_topics = fp.readlines()
excluded_set = set()
for topic in tmp_topics:
    excluded_set.add(topic.strip())

with open(nolemma_list_file, 'rt', encoding='utf8') as fp:
        topics_file = fp.readlines()

topics_final = []
topics_lemma = []

excluded = 0
for topic in topics_file:
    topic_orig = topic.strip()
    if topic_orig in excluded_set:
        print("=== EXCLUDE ", topic_orig)
        excluded += 1
        continue
    topic = re.sub('\W',' ',topic)
    topic = re.sub('\ +',' ',topic)
    doc = nlp(topic.strip().lower())
    lemmas = []

    for token in doc:
        if token.is_stop:
            continue
        lemmas.append(token.lemma_)    
    topics_lemma.append("_".join(lemmas))
    topics_final.append(topic_orig)

    print(topic_orig, ' --- ', "_".join(lemmas))

print('Excluded: ', excluded)
print('Final topics: ', len(topics_lemma), len(topics_final))
    
with open(lemma_list_file_excluded, 'wt', encoding='utf8') as f:
    for topic_lemma in topics_lemma:
        f.write(topic_lemma + "\n")
with open(nolemma_list_file_excluded, 'wt', encoding='utf8') as f:
    for topic_final in topics_final:
        f.write(topic_final + "\n")  

plaatselijke ontwikkeling  ---  plaatselijk_ontwikkeling
plattelandsontwikkeling  ---  plattelandsontwikkeling
beheer van de waterbronnen  ---  beheer_waterbron
opleidingsniveau  ---  opleidingsniveau
algemeen onderwijs  ---  algemeen_onderwijs
bank  ---  bank
wet van de Europese Gemeenschap  ---  wet_europees_gemeenschap
cultureel erfgoed  ---  cultureel_erfgoed
landbouwgrond  ---  landbouwgrond
landbouwlandschap  ---  landbouwlandschap
landbouwproductie  ---  landbouwproductie
economische ontwikkeling  ---  economisch_ontwikkeling
voedselketen  ---  voedselketen
voedingsmiddelenindustrie  ---  voedingsmiddelenindustrie
voedselkwaliteit  ---  voedselkwaliteit
wereldwijde opwarming  ---  wereldwijd_opwarming
broeikaseffect  ---  broeikaseffect
industriële ontwikkeling  ---  industrieel_ontwikkeling
arbeidsmarkt  ---  arbeidsmarkt
toegang tot land  ---  toegang_land
ruilverkaveling  ---  ruilverkaveling
landinrichting  ---  landinrichting
landschapsarchitectuur  ---  landschapsarchitect

achteruitgang van het ecosysteem  ---  achteruitgang_ecosysteem
kledingindustrie  ---  kledingindustrie
aardewerkindustrie  ---  aardewerkindustrie
markteconomie  ---  markteconomie
milieubeheer  ---  milieubeheer
softwareontwikkeling  ---  softwareontwikkeling
=== EXCLUDE  landbouwopslag
akkerbouw  ---  akkerbouw
landbouwproduct  ---  landbouwproduct
klimaatschommeling  ---  klimaatschommeling
landbouweconomie  ---  landbouweconomie
landinrichting  ---  landinrichting
industriële planning  ---  industrieel_planning
economische planning  ---  economisch_planning
architectuur  ---  architectuur
=== EXCLUDE  voedselhandel
bodemtextuur  ---  bodemtextuur
voedseltechnologie  ---  voedseltechnologie
milieuplanning  ---  milieuplanning
landbouwtechniek  ---  landbouwtechniek
zuivelindustrie  ---  zuivelindustrie
regenwater  ---  regenwater
gronddruk  ---  gronddruk
farmaceutische industrie  ---  farmaceutisch_industrie
=== EXCLUDE  dierenhuisvesting
bevolkingsstructuur  ---  bevolkingsstruct

zuivering door de grond  ---  zuivering_grond
grondverbetering  ---  grondverbetering
mariene geologie  ---  marien_geologie
beschermingssysteem  ---  beschermingssysteem
industriële processen  ---  industrieel_proces
afbraak van verontreinigende stoffen  ---  afbraak_verontreinigen_stof
energie-industrie  ---  energie_industrie
milieu-industrie  ---  milieu_industrie
afwateren  ---  afwateren
inkomen  ---  inkomen
=== EXCLUDE  plaatsbescherming
rivierbeheer  ---  rivierbeheer
wind  ---  wind
vrij toegankelijke informatie  ---  toegankelijk_informatie
=== EXCLUDE  bodemproces
proceswater  ---  proceswater
watervoorziening  ---  watervoorziening
industriegebied  ---  industriegebied
opberging te land  ---  opberging_land
warmte (fysica)  ---  warmt_fysica
ecologische (levens)gemeenschap  ---  ecologisch_leven_gemeenschap
personenvervoer  ---  personenvervoer
inschatting van (natuurlijke) hulpbronnen  ---  inschatting_natuurlijk_hulpbron
het opstellen van economische prognoses  ---  opst

marketing  ---  marketing
personenwagen  ---  personenwagen
landbouwmachine  ---  landbouwmachine
gezondheidswetgeving  ---  gezondheidswetgeving
=== EXCLUDE  mijnbouwvoorschrift
afvalwater  ---  afvalwater
overheidsopdracht  ---  overheidsopdracht
goederen en diensten  ---  goed_dienst
bodemlaag  ---  bodemlaag
industriële economie  ---  industrieel_economie
=== EXCLUDE  landecologie
sociale omstandigheden  ---  sociaal_omstandigheid
waterdier  ---  waterdier
biologische (hulp)bronnen  ---  biologisch_hulp_bronnen
leefomgeving  ---  leefomgeving
sociale kosten  ---  sociaal_kost
markt  ---  markt
grondwaarde  ---  grondwaaren
werkgelegenheid  ---  werkgelegenheid
privaat domein  ---  privaat_domein
staatsdomein  ---  staatsdomein
=== EXCLUDE  vacuümindustrie
dierlijke erfelijkheidsleer  ---  dierlijk_erfelijkheidsleer
bouwtechnologie  ---  bouwtechnologie
onderwijsplanning  ---  onderwijsplanning
nieuwe gemeente  ---  nieuw_gemeente
ruimtelijke verdeling  ---  ruimtelijk_verdeling
bes

arbeidsrecht  ---  arbeidsrecht
=== EXCLUDE  milieuterminologie
nederzetting in het platteland  ---  nederzetting_platteland
Arctisch gebied  ---  arctisch_gebied
geschiedkundige evolutie  ---  geschiedkundig_evolutie
milieu-investering  ---  milieu_investering
prioriteit op milieugebied  ---  prioriteit_milieugebied
milieugegevens  ---  milieugegeven
natuurramp  ---  natuurramp
=== EXCLUDE  milieustimuleringsmaatregel
nederzetting  ---  nederzetting
planologische maatregel  ---  planologisch_maatregel
dierengedrag  ---  dierengedrag
marktonderzoek  ---  marktonderzoek
=== EXCLUDE  beroepsgroenteteelt
=== EXCLUDE  plantenstand
demineralisatie van water  ---  demineralisatie_water
wetshandhaving  ---  wetshandhaving
jeugdwerk  ---  jeugdwerk
uitvoeringswet  ---  uitvoeringswet
=== EXCLUDE  milieuangst
openbare instelling  ---  openbaar_instelling
steenkoolindustrie  ---  steenkoolindustrie
industrieel slib  ---  industrieel_slib
gemeenschap  ---  gemeenschap
sociale analyse  ---  sociaa

...

monopolie  ---  monopolie
rook  ---  rook
verlies  ---  verlies
kamperen  ---  kamperen
hydrobiologie  ---  hydrobiologie
weekdieren  ---  weekdier
octrooi  ---  octrooi
herinvoering  ---  herinvoering
=== EXCLUDE  afvalinvoervergunning
nitrieten  ---  nitriet
vergassing  ---  vergassing
draagvermogen  ---  draagvermogen
primaten  ---  primaat
het per ongeluk vrijkomen van organismen  ---  per_ongeluk_vrijkom_organisme
ongevallen  ---  ongeval
radioactieve vervuiler  ---  radioactief_vervuiler
=== EXCLUDE  zoutlading
kanaalsluis  ---  kanaalsluis
storten van afval  ---  storten_afval
bijen  ---  bij
smeermiddel  ---  smeermiddel
oorlog  ---  oorlog
iets in voorraad houden  ---  voorraad_houden
vlees  ---  vlees
ergonomie  ---  ergonomie
botanische tuin  ---  botanisch_tuin
puntbron  ---  puntbron
koper  ---  koper
sedimentologie  ---  sedimentologie
kruising  ---  kruising
toezicht  ---  toezicht
oude afzetting  ---  oud_afzetting
wintersportoord  ---  wintersportoord
=== EXCLUDE  fosf

verbrandingsrest  ---  verbrandingsrest
=== EXCLUDE  emulgatie
=== EXCLUDE  anti-vervuilingsaanmoediging
freon  ---  freon
analyseren  ---  analyser
document  ---  document
vakantiekamp  ---  vakantiekamp
kamp  ---  kamp
allergeen  ---  allergeen
mijten  ---  mijt
handelsbarrière  ---  handelsbarrière
pedosfeer  ---  pedosfeer
zoutzuur  ---  zoutzuur
afvalstortplaats  ---  afvalstortplaat
zaagsel  ---  zaagsel
krijt  ---  krijt
strontium  ---  strontium
thermodynamica  ---  thermodynamica
misdadig gedrag  ---  misdadig_gedrag
paard  ---  paard
schietbaan  ---  schietbaan
(steen)groeve  ---  ste_groeve
een steengroeve ontginnen  ---  steengroev_ontginn
viaduct  ---  viaduct
wortel  ---  wortel
opname  ---  opname
parameters  ---  parameter
chloride  ---  chloride
stemming  ---  stemming
codificatie  ---  codificatie
eiken(bomen)  ---  eik_boom
geurstof  ---  geurstof
plantenziektekunde  ---  plantenziektekunen
=== EXCLUDE  schadeloosstellingseis
voorspelling  ---  voorspelling
osmose  -

beeldpunt  ---  beeldpunt
zuiverend vermogen  ---  zuiveren_vermogen
dwangmaatregel  ---  dwangmaatregel
expediteur  ---  expediteur
elektrolyse  ---  elektrolyse
congres  ---  congres
pendelen  ---  pendel
virologie  ---  virologie
etsende stoffen  ---  etsen_stof
aanbeveling  ---  aanbeveling
=== EXCLUDE  strooiapparaat (v. stalmest of tuinafval)
=== EXCLUDE  beeldclassificatie onder toezicht
fluor  ---  fluor
klif  ---  klif
=== EXCLUDE  afvalstatistieken
rechtszaak  ---  rechtszaak
resolutie  ---  resolutie
onderhandelbare heffingen  ---  onderhandelbar_heffing
vogelkunde  ---  vogelkunen
=== EXCLUDE  chloorfluorkoolstof
marterachtigen  ---  marterachtigen
ontsmettingsmiddel  ---  ontsmettingsmiddel
gehoor (zintuig)  ---  gehoor_zintuig
bezwaar  ---  bezwaar
asbest  ---  asbest
kikkers  ---  kikker
naaktzadigen  ---  naaktzadig
verrijken  ---  verrijk
elasticiteit  ---  elasticiteit
stolling  ---  stolling
dioxine  ---  dioxine
(door)weken  ---  week
virussen  ---  virus
eigenschap

basidiomycete  ---  basidiomycete
(ruwe) aardolie  ---  ruw_aardolie
normalisatie  ---  normalisatie
hondachtige  ---  hondachtig
vergif  ---  vergif
weekmaker  ---  weekmak
=== EXCLUDE  verambtelijking
=== EXCLUDE  polychloordibenzo-p-dioxine
leer van het bloed  ---  leer_bloed
oorwurmen  ---  oorwurm
water rijk aan rottend organisch materiaal  ---  water_rijk_rotten_organisch_materiaal
herhuisvesting  ---  herhuisvesting
=== EXCLUDE  polybroombifenyl
polychloorbifenyl  ---  polychloorbifenyl
bibliografie  ---  bibliografie
=== EXCLUDE  bederfwerend middel
stranden  ---  strand
afschrijven  ---  afschrijven
reuk  ---  reuk
statistiek  ---  statistiek
=== EXCLUDE  algendodende stof
gegevens met geografische componenten  ---  gegeven_geografisch_component
DDT  ---  ddt
encyclopedie  ---  encyclopedie
dendrochronologie  ---  dendrochronologie
wantsen  ---  wants
colorimetrie  ---  colorimetrie
cyanaat  ---  cyanaat
vulcaniseren  ---  vulcaniseren
purine  ---  purine
norm  ---  norm
(ver)

Ursidae  ---  ursidae
bioreactor  ---  bioreactor
bioveiligheid  ---  bioveiligheid
bron  ---  bron
boekhouding  ---  boekhouding
Excluded:  727
Final topics:  4581 4581


In [30]:
# adding all the words and topics to model

# load the lemma list
with open(lemma_list_file_excluded) as fp:
    topics = fp.readlines()
# load the nolemma list
with open(nolemma_list_file_excluded) as fp:
    topics_nolemma = fp.readlines()    
    

#try if the topic-lemma words exist in the model     
new_vectors = []
new_words = []
for i, (topic, topic_nolemma) in enumerate(zip(topics, topics_nolemma)):
#for i,topic in enumerate(topics):
    topic_nolemma = re.sub('\W',' ',topic_nolemma)
    topic_nolemma = re.sub('\ +',' ',topic_nolemma)
    doc = nlp(topic_nolemma.strip().lower())
    nolemmas = []
    for token in doc:
        if token.is_stop:
            continue
        else:
            nolemmas.append(token.text) 
            
    topic = topic.strip()
    topic_words = topic.split('_')
    
    if len(topic_words) != len(nolemmas):
        print("!!!!! error - not equal length: ", nolemmas, topic_words)
        continue
    #if len(topic_words)<2:
    #    continue
    
    
    tmp_vectors = []
    
    for word, word_nolemma in zip(topic_words, nolemmas):
    #for word in topic_words:
        vector = get_vector(word)
        
        try:
            tmp_vectors.append(model[word])
        except: 
            print('\nNOT in vocabulary: ', word, topic)
            vector = get_vector(word)
            if vector != []:
                tmp_vectors.append(vector)
                if word not in new_words:
                    print('append ', word)
                    new_vectors.append(vector)
                    new_words.append(word)   
            else:
                vector = get_vector(word_nolemma)
                if vector != []:
                    tmp_vectors.append(vector)
                    if word not in new_words:
                        print('append ', word)
                        new_vectors.append(vector)
                        new_words.append(word)  
                else:
                    raise Warning(f"XXX {word} {topic} {topic_nolemma}")
                    print("XXX", word, topic, topic_nolemma) 
    #if word not in new_words:
    #    new_vectors.append(list(np.average(tmp_vectors, axis=0)))
    #    new_words.append(word)        
    if len(tmp_vectors) > 1:
        if topic not in new_words:
   
            print('append ', topic)
            new_vectors.append(list(np.average(tmp_vectors, axis=0)))
            new_words.append(topic)
        else:
            print('doubled TOPIC: ', topic)    
    else:
        print('--- only one word, not a new topic>', topic)

append  plaatselijk_ontwikkeling
--- only one word, not a new topic> plattelandsontwikkeling
append  beheer_waterbron
--- only one word, not a new topic> opleidingsniveau
append  algemeen_onderwijs
--- only one word, not a new topic> bank
append  wet_europees_gemeenschap
append  cultureel_erfgoed
--- only one word, not a new topic> landbouwgrond

NOT in vocabulary:  landbouwlandschap landbouwlandschap
append  landbouwlandschap
--- only one word, not a new topic> landbouwlandschap
--- only one word, not a new topic> landbouwproductie
append  economisch_ontwikkeling
--- only one word, not a new topic> voedselketen
--- only one word, not a new topic> voedingsmiddelenindustrie
--- only one word, not a new topic> voedselkwaliteit
append  wereldwijd_opwarming
--- only one word, not a new topic> broeikaseffect
append  industrieel_ontwikkeling
--- only one word, not a new topic> arbeidsmarkt
append  toegang_land
--- only one word, not a new topic> ruilverkaveling
--- only one word, not a new t

/home/dzon/kajo/kajo_semexp_core/lib/python3.6/site-packages/ipykernel_launcher.py:47: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.



NOT in vocabulary:  bodemdegradatie bodemdegradatie
append  bodemdegradatie
--- only one word, not a new topic> bodemdegradatie
--- only one word, not a new topic> bodemerosie
append  duurzaam_ontwikkeling
append  stedelijk_planning

NOT in vocabulary:  waterlichaam waterlichaam
append  waterlichaam
--- only one word, not a new topic> waterlichaam
--- only one word, not a new topic> watererosie
--- only one word, not a new topic> waterbeheer

NOT in vocabulary:  winderosie winderosie
append  winderosie
--- only one word, not a new topic> winderosie
append  marien_ecosysteem
append  ecosysteem_water
--- only one word, not a new topic> waterweg
--- only one word, not a new topic> beroepsbevolking
--- only one word, not a new topic> klimaatverandering
append  technologisch_verandering

NOT in vocabulary:  milieuverandering milieuverandering
append  milieuverandering
--- only one word, not a new topic> milieuverandering

NOT in vocabulary:  landbeheer landbeheer_inrichting
append  landbeh

--- only one word, not a new topic> stadslandschap
append  nieuw_installatie
--- only one word, not a new topic> toerisme
append  landbouw_milieu
append  biologisch_productie
append  milieu_ontwikkeling
--- only one word, not a new topic> bodemschatten
--- only one word, not a new topic> beheer
--- only one word, not a new topic> beleid

NOT in vocabulary:  bosexploitatie bosexploitatie
append  bosexploitatie
--- only one word, not a new topic> bosexploitatie
--- only one word, not a new topic> vervoerssysteem
append  zoutgehalte_bodem
append  zoutgehalte_water

NOT in vocabulary:  ontwikkelingsplanning ontwikkelingsplanning
append  ontwikkelingsplanning
--- only one word, not a new topic> ontwikkelingsplanning

NOT in vocabulary:  vervoersplanning vervoersplanning
append  vervoersplanning
--- only one word, not a new topic> vervoersplanning
append  landbouw_huis_dier
append  industrieel_wetgeving
append  financieel_hulp
--- only one word, not a new topic> milieukwaliteit
--- only one 

append  zuivering_grond
--- only one word, not a new topic> grondverbetering
append  marien_geologie

NOT in vocabulary:  beschermingssysteem beschermingssysteem
append  beschermingssysteem
--- only one word, not a new topic> beschermingssysteem
append  industrieel_proces
append  afbraak_verontreinigen_stof
append  energie_industrie
append  milieu_industrie
--- only one word, not a new topic> afwateren
--- only one word, not a new topic> inkomen

NOT in vocabulary:  rivierbeheer rivierbeheer
append  rivierbeheer
--- only one word, not a new topic> rivierbeheer
--- only one word, not a new topic> wind
append  toegankelijk_informatie
--- only one word, not a new topic> proceswater
--- only one word, not a new topic> watervoorziening
--- only one word, not a new topic> industriegebied
append  opberging_land

NOT in vocabulary:  warmt warmt_fysica
append  warmt
append  warmt_fysica
append  ecologisch_leven_gemeenschap
--- only one word, not a new topic> personenvervoer
append  inschatting_

...

--- only one word, not a new topic> vonnis
--- only one word, not a new topic> alarm
append  lossen_hout
--- only one word, not a new topic> berekening
--- only one word, not a new topic> besluit
--- only one word, not a new topic> verbrandingsmotor

NOT in vocabulary:  metalen metalen
--- only one word, not a new topic> metalen
--- only one word, not a new topic> laboratoriumproef
--- only one word, not a new topic> tuin
--- only one word, not a new topic> verfwinkel

NOT in vocabulary:  overheidsdaad onrechtmatig_overheidsdaad
append  overheidsdaad
append  onrechtmatig_overheidsdaad
append  verbod_verspreiding_afvalwater
append  aluminium_bestaan_container_houder
--- only one word, not a new topic> soort
--- only one word, not a new topic> nomade
--- only one word, not a new topic> bruinkoolwinning
append  herintroductie_plantensoort
--- only one word, not a new topic> ivoor
--- only one word, not a new topic> smog
append  regering_kabinet
append  indexeren_informatie
append  aantal_

--- only one word, not a new topic> woordenboek

NOT in vocabulary:  ijzerschroot ijzerschroot
append  ijzerschroot
--- only one word, not a new topic> ijzerschroot
--- only one word, not a new topic> heffing
--- only one word, not a new topic> autoclaaf
--- only one word, not a new topic> salmonella
--- only one word, not a new topic> vanadium
--- only one word, not a new topic> geluid
--- only one word, not a new topic> classificatie
--- only one word, not a new topic> antibioticum

NOT in vocabulary:  flotatie flotatie
append  flotatie
--- only one word, not a new topic> flotatie
--- only one word, not a new topic> leukemie
append  uiterlijk_kenmerk
--- only one word, not a new topic> licht
--- only one word, not a new topic> hagedis
--- only one word, not a new topic> kantoor
--- only one word, not a new topic> uitspoeling
--- only one word, not a new topic> looistof
append  ver_meng
append  plaats_ophogen
--- only one word, not a new topic> kalk
append  gemakkelijk_afbreekbaar_sto

--- only one word, not a new topic> expediteur
--- only one word, not a new topic> elektrolyse
--- only one word, not a new topic> congres
--- only one word, not a new topic> pendel
--- only one word, not a new topic> virologie
append  etsen_stof
--- only one word, not a new topic> aanbeveling
--- only one word, not a new topic> fluor
--- only one word, not a new topic> klif
--- only one word, not a new topic> rechtszaak
--- only one word, not a new topic> resolutie

NOT in vocabulary:  onderhandelbar onderhandelbar_heffing
append  onderhandelbar
append  onderhandelbar_heffing

NOT in vocabulary:  vogelkunen vogelkunen
append  vogelkunen
--- only one word, not a new topic> vogelkunen
--- only one word, not a new topic> marterachtigen
--- only one word, not a new topic> ontsmettingsmiddel
append  gehoor_zintuig
--- only one word, not a new topic> bezwaar
--- only one word, not a new topic> asbest
--- only one word, not a new topic> kikker
--- only one word, not a new topic> naaktzadig
-

--- only one word, not a new topic> salamander
--- only one word, not a new topic> radon

NOT in vocabulary:  ontzwaveling ontzwaveling
--- only one word, not a new topic> ontzwaveling
--- only one word, not a new topic> buideldieren
--- only one word, not a new topic> vlo

NOT in vocabulary:  basiciteit basiciteit
append  basiciteit
--- only one word, not a new topic> basiciteit

NOT in vocabulary:  skihelling skihelling
append  skihelling
--- only one word, not a new topic> skihelling

NOT in vocabulary:  spasmodisch spasmodisch_kroep
append  spasmodisch
append  spasmodisch_kroep
--- only one word, not a new topic> silo
--- only one word, not a new topic> viscositeit
--- only one word, not a new topic> mycologie
append  zone_vallen_recht_voorkoop
--- only one word, not a new topic> steroïd
--- only one word, not a new topic> foto

NOT in vocabulary:  limnologie stratificatie_omkering_limnologie
append  stratificatie_omkering_limnologie

NOT in vocabulary:  anthropologisch anthropolog


NOT in vocabulary:  milieuactieprogramma milieuactieprogramma
append  milieuactieprogramma
--- only one word, not a new topic> milieuactieprogramma

NOT in vocabulary:  milieuvoetafdruk milieuvoetafdruk
append  milieuvoetafdruk
--- only one word, not a new topic> milieuvoetafdruk
append  ecologisch_bestuur
--- only one word, not a new topic> milieuvoorschrift

NOT in vocabulary:  milieugebruiksruimte milieugebruiksruimte
append  milieugebruiksruimte
--- only one word, not a new topic> milieugebruiksruimte
append  eu_regeling_emissiehandel
append  behoud_ex_situ
--- only one word, not a new topic> overstromingsgevaar

NOT in vocabulary:  voedselafval voedselafval
append  voedselafval
--- only one word, not a new topic> voedselafval
append  bebossen_gebied
append  verbruik_fossiel_brandstof
append  functioneel_voeding
append  groen_economie
--- only one word, not a new topic> broeikasgasemissie
doubled TOPIC:  bovengronds_biomassa
--- only one word, not a new topic> habitatrichtlijn
app

In [31]:
# add topics to model
model.add(new_words,new_vectors)

In [40]:
# save the model with new topics
model.save_word2vec_format(excluded_model_name, binary=True)
model = KeyedVectors.load_word2vec_format(excluded_model_name, binary=True)

In [43]:
# try the model
model.similar_by_vector('gemoed')

[('bemoederen', 0.5206096172332764),
 ('bedaren', 0.5075072050094604),
 ('laaien', 0.4725201725959778),
 ('oplaaien', 0.4684869349002838),
 ('verbroederen', 0.4457176625728607),
 ('beroering', 0.44479846954345703),
 ('hitsen', 0.4322381019592285),
 ('verhitten', 0.4244909882545471),
 ('sussen', 0.4167637825012207),
 ('roering', 0.4148557782173157)]

In [44]:
# find most similar to topics to terms in the new model

# load the new lemma list
with open(lemma_list_file_excluded) as fp:
    topics = fp.readlines()

for i, topic in enumerate(topics):
    topic = topic.strip()
    print('\n-----\n', topic)
    try:
        print(i, model.similar_by_vector(topic))
    except:
        print(topic, 'NOT in model')


-----
 plaatselijk_ontwikkeling
0 [('plaatselijk', 0.8662415742874146), ('plaatselijk_vrijetijdsbesteding', 0.8270551562309265), ('plaatselijk_overheid', 0.8134081959724426), ('plaatselijk_bouwmateriaal', 0.7971285581588745), ('beleid_plaatselijk_overheid', 0.7902386784553528), ('plaatselijk_warmtevoorziening', 0.7807294130325317), ('plannen_stedelijk_ontwikkeling', 0.7750746011734009), ('plaatselijk_financiën', 0.7685471177101135), ('plaatselijk_verkeer', 0.7548452615737915), ('cultureel_ontwikkeling', 0.7546594738960266)]

-----
 plattelandsontwikkeling
1 [('plattelandsbeleid', 0.7567116618156433), ('platteland', 0.6597249507904053), ('ontwikkelingssamenwerking', 0.642474889755249), ('biotechnologie_landbouw', 0.6391189098358154), ('milieuwetgeving_betrekking_landbouw', 0.6330200433731079), ('landbouw_milieu', 0.6286821365356445), ('invloed_landbouw_milieu', 0.6279362440109253), ('landmaatschappij', 0.6250737309455872), ('intensief_landbouw', 0.6238653659820557), ('landbouw', 0.6203

22 [('landschapsarchitect', 0.8181471824645996), ('tuinarchitectuur', 0.7134948968887329), ('landschapsinrichting', 0.7095366716384888), ('interieurarchitectuur', 0.688307523727417), ('landschapsontwikkeling', 0.6558452844619751), ('stedenbouw', 0.6389328837394714), ('binnenhuisarchitectuur', 0.6251627206802368), ('landschapsecologie', 0.6125889420509338), ('architectuur', 0.6074659824371338), ('tuinkunst', 0.6008426547050476)]

-----
 landschapsplanning
23 [('landschapsecologie', 0.6183904409408569), ('ontwikkelingsplanning', 0.6174561977386475), ('landschapszorg', 0.6094745397567749), ('landschapsontwikkeling', 0.6006873250007629), ('vervoersplanning', 0.5889133810997009), ('onderwijsplanning', 0.5879173278808594), ('cultuurlandschap', 0.5775154232978821), ('landbouwlandschap', 0.5752402544021606), ('planning_kustzone', 0.5722979307174683), ('rivierenlandschap', 0.5702264308929443)]

-----
 grondgebruik
24 [('landgebruik', 0.7845229506492615), ('ruimtegebruik', 0.7012342810630798), (

42 [('erosie', 0.713876485824585), ('winderosie', 0.6975604295730591), ('bodemerosie', 0.6437233686447144), ('kusterosie', 0.6255168318748474), ('grondwaterstroming', 0.6191037893295288), ('waterstroming', 0.6095383167266846), ('verwering', 0.5963258147239685), ('bodemvorming', 0.5960437655448914), ('sedimentatie', 0.5892895460128784), ('grondwaterwinning', 0.5775061249732971)]

-----
 waterbeheer
43 [('gemeentelijk_waterbeheer', 0.8819676637649536), ('grondwaterbeheer', 0.8639160394668579), ('waterbeheersing', 0.8045111894607544), ('waterbeheerder', 0.7769753336906433), ('watersysteem', 0.762550950050354), ('waterschap', 0.7395204305648804), ('waterkwaliteit', 0.7358644008636475), ('waterbeheerplan', 0.7325818538665771), ('waterbeleid', 0.7228801250457764), ('ecologie_water', 0.7206017374992371)]

-----
 winderosie
44 [('erosie', 0.7316945791244507), ('watererosie', 0.6975603699684143), ('bodemerosie', 0.6713078022003174), ('tanderosie', 0.644499659538269), ('kusterosie', 0.6160998344

62 [('landschapsbeheer', 0.8216711282730103), ('natuurbeheerder', 0.7544478178024292), ('natuurontwikkeling', 0.7361350655555725), ('organisatie_natuurbehoud', 0.7297457456588745), ('natuurbehoud', 0.7191441655158997), ('natuurbeleid', 0.7023710012435913), ('terreinbeheer', 0.69813072681427), ('weidevogelbeheer', 0.6962774395942688), ('duurzaam_bosbeheer', 0.6883149743080139), ('bosbeheer', 0.6873830556869507)]

-----
 bescherming_natuurgebied
63 [('natuurgebied', 0.7950560450553894), ('bescherming', 0.7762265801429749), ('beschermen_landschap', 0.7721781730651855), ('gevoelig_natuurgebied', 0.7631096839904785), ('bescherming_stroomgebied', 0.7552374005317688), ('beschermen_gebied', 0.7327544689178467), ('bescherming_overstroming', 0.7291991114616394), ('beschermen_stroomgebied', 0.7077969312667847), ('beschermen', 0.6843119859695435), ('beschermingsgebied', 0.6813859343528748)]

-----
 organisatie_natuurbehoud
64 [('natuurbehoud', 0.8870013952255249), ('beleid_natuurbehoud', 0.8131250

81 [('overblijven_hoeveelheid_water', 0.7848765850067139), ('water_territoriaal_eigendomsrecht_vallen', 0.7831805944442749), ('bedreiging_water', 0.7747209668159485), ('voorraad_natuurlijk_hulpbron', 0.7677240967750549), ('gebruiken_water', 0.764919102191925), ('wetgeving_vervuiling', 0.7642806768417358), ('water_industrieel_gebruik', 0.7638999223709106), ('organisme_water', 0.7625914216041565), ('wetgeving_betrekking_natuur', 0.7615518569946289), ('hergebruik_water', 0.7611236572265625)]

-----
 mens_veroorzaken_klimaatsverandering
82 [('mens_veroorzaken_ramp', 0.8644546866416931), ('mens', 0.8566707968711853), ('verhouding_mens_natuur', 0.8162909746170044), ('effect_mens', 0.8144419193267822), ('aandoening_milieu_maken', 0.7367458343505859), ('menselijk_ziekte', 0.7079221606254578), ('ecologisch_leven_gemeenschap', 0.7071527242660522), ('dier_algemeen_systematisch', 0.6966469287872314), ('bescherming_wild_leven_plant_dier', 0.6964806914329529), ('leefomgeving_wild_leven_plant_dier', 

...

4486 [('gevolg', 0.9262756705284119), ('gevolg_milieuverontreiniging', 0.9202184677124023), ('oorzaak_gevolg_relatie', 0.8358011245727539), ('aanpassing_klimaatverandering', 0.7587807178497314), ('klimaatverandering', 0.7524397969245911), ('oorzaak', 0.7338491678237915), ('schade_gevolg_militair_oefening', 0.7333633303642273), ('invloed_verandering_klimaat', 0.7308892011642456), ('gevolg_lawaai_gezondheid', 0.7215567231178284), ('oorzaak_ongeluk', 0.718878984451294)]

-----
 klimaatbeleid
4487 [('energiebeleid', 0.7124946713447571), ('klimaatplan', 0.6959789991378784), ('klimaatprogramma', 0.6916925311088562), ('klimaatdoelstelling', 0.6808154582977295), ('klimaatactieplan', 0.6800957322120667), ('klimaatdoel', 0.6680597066879272), ('klimaatactie', 0.6622835397720337), ('klimaatverdrag', 0.6548985242843628), ('klimaatambitie', 0.6497784852981567), ('natuurbeleid', 0.6441910266876221)]

-----
 klimaatzone
4488 [('woudgebied_matigen_klimaatzone', 0.802932620048523), ('bos_matigen_klimaat

4506 [('eu', 0.9723073244094849), ('eg_verdrag', 0.6247749924659729), ('eg_beleid', 0.6173425912857056), ('eg_verordening', 0.6021721959114075), ('eg_richtlijn_afvalverwerking', 0.5964792370796204), ('eg_richtlijn', 0.5964369773864746), ('nl', 0.5951776504516602), ('eg_richtlijn_verpakking', 0.593039333820343), ('eg_richtlijn_biocide', 0.5926189422607422), ('ec', 0.5925281643867493)]

-----
 behoud_ex_situ
4507 [('ex', 0.8665891885757446), ('situ', 0.7021540403366089), ('aequo', 0.5753904581069946), ('cur', 0.5701894164085388), ('opl', 0.5427542328834534), ('nihilo', 0.5375293493270874), ('voto', 0.5370307564735413), ('ea', 0.533001184463501), ('re', 0.5328917503356934), ('element_groep_iv', 0.5231747627258301)]

-----
 overstromingsgevaar
4508 [('overstromingsrisico', 0.7405620813369751), ('overstroming', 0.7401817440986633), ('overstromingsgebied', 0.7129985690116882), ('overstromingsgevoelig', 0.6947077512741089), ('wateroverla', 0.6894090175628662), ('bescherming_overstroming', 0.6

4526 [('stedelijk_afvalwater', 0.900304913520813), ('huishoudelijk_afvalwater', 0.8864898085594177), ('afvalwater', 0.8827752470970154), ('industrieel_afvalwater', 0.8541948199272156), ('gemeentelijk_watervoorziening', 0.8428689241409302), ('organisch_afvalwater', 0.8280431032180786), ('gemeentelijk_waterbeheer', 0.8252485990524292), ('combineren_afvalwater', 0.8154471516609192), ('gemeentelijk_afval', 0.806013822555542), ('afvalwaterzuivering', 0.7958106994628906)]

-----
 nitraatrichtlijn
4527 [('nitraat', 0.6457894444465637), ('habitatrichtlijn', 0.6078662872314453), ('kaderrichtlijn', 0.6056349277496338), ('meststoffenwet', 0.6003507971763611), ('mestwetgeving', 0.5753827691078186), ('bemestingsnorm', 0.5719590187072754), ('habitatrichtlijngebied', 0.5604986548423767), ('mestbeleid', 0.5539541840553284), ('landbouwwetgeving', 0.5451147556304932), ('richtlijn_waterkwaliteit', 0.5450989007949829)]

-----
 organisch_bodemverbeteraar
4528 [('organisch', 0.8985841870307922), ('organisch

4548 [('landbouw', 0.8544491529464722), ('biotechnologie_landbouw', 0.843787431716919), ('voedselproductie_landbouw', 0.8289111256599426), ('landbouw_veehouderij', 0.8283773064613342), ('milieuwetgeving_betrekking_landbouw', 0.8084189295768738), ('intensief_landbouw', 0.8083142042160034), ('landbouw_milieu', 0.786211371421814), ('biologisch_landbouw', 0.7714463472366333), ('meteorologie', 0.7666175961494446), ('invloed_landbouw_milieu', 0.7595441341400146)]

-----
 ecologisch_ontwikkeling
4549 [('ecologisch', 0.905320405960083), ('ecologisch_inventarisatie', 0.8833140134811401), ('ecologisch_aanpassing', 0.8635703921318054), ('economisch_ecologisch_efficiëntie', 0.8633954524993896), ('ecologisch_ongelijkheid', 0.8610144257545471), ('ecologisch_voetafdruk', 0.8510012626647949), ('biologisch_ontwikkeling', 0.8491066694259644), ('ecologisch_evenwicht', 0.8466533422470093), ('technologisch_ontwikkeling', 0.8350939154624939), ('ecologisch_beschaving', 0.8246484398841858)]

-----
 toekennen_

4571 [('spons', 0.6765835285186768), ('sponsen', 0.6182262897491455), ('glasspons', 0.5874707102775574), ('schuurspons', 0.5554639101028442), ('beinkt', 0.5512620210647583), ('distressinkt', 0.5478513240814209), ('brusho', 0.5460612773895264), ('panpastel', 0.5429327487945557), ('versafine', 0.5426872968673706), ('stempelen', 0.5308207273483276)]

-----
 inventarisatie
4572 [('inventarisering', 0.7862417697906494), ('inventariseren', 0.7689880132675171), ('ecologisch_inventarisatie', 0.7563105225563049), ('inventarisatielijst', 0.7522665858268738), ('inventariser', 0.7337489128112793), ('asbestinventarisatie', 0.7237100005149841), ('inventarisatiefase', 0.6930948495864868), ('risicoinventarisatie', 0.6848450899124146), ('quickscan', 0.6574012637138367), ('inventarisatieformulier', 0.6566029787063599)]

-----
 bio_ethiek
4573 [('bio', 0.9457381963729858), ('bio_indicator', 0.942329466342926), ('bio_monitoring', 0.939181923866272), ('bio_accumulatie', 0.9346104264259338), ('productie_bio